In [2]:
import pandas as pd

question_df = pd.read_csv('../data/problem_descriptions.csv')
question_df.head()

,problem_id,problem_description,language
0,p02872,Problem Setting\r\nOverview\r\nSpecification o...,english
1,p01327,Problem G: 天体観測\r\n2012年のある日のこと。\r\n北の田舎町に住むあな...,japanese
2,p02871,Problem Setting\r\nOverview\r\nSpecification o...,english
3,p01482,Memory Leak\r\nTime Limit: 8 sec / Memory Limi...,japanese
4,p01535,Markup language has Declined\r\nE: マークアップ言語は衰退...,japanese


In [3]:
problem_dict = {}
for index, row in question_df.iterrows():
    problem_dict[row['problem_id']] = row['problem_description']

In [16]:
len(problem_dict)

3999

In [4]:
q_df = pd.read_csv('../data/Python/python_problem_kc_matrix.csv')
q_df.head()

,id,KC_1,KC_2,KC_3,KC_4,KC_5,KC_6,KC_7,KC_8,KC_9,...,KC_69,KC_70,KC_71,KC_72,KC_73,KC_74,KC_75,KC_76,KC_77,KC_78
0,p00000,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,p00001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,p00002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,p00003,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,p00004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
question_list = {}
for index, row in q_df.iterrows():
    question_list[index] = problem_dict[row['id']]
print(question_list[0])
len(question_list)


QQ
Write a program which prints multiplication tables in the following format:
Input
No input.
Output
Template for C
Template for C++
Template for Java


2233

In [6]:
question_list[0]

'QQ\r\nWrite a program which prints multiplication tables in the following format:\r\nInput\r\nNo input.\r\nOutput\r\nTemplate for C\r\nTemplate for C++\r\nTemplate for Java'

In [7]:
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import torch
import torch.nn as nn


tokenizer = AutoTokenizer.from_pretrained("../graphcodebert-base")  # 使用的是本地的codebert-base 或 graphcodebert-base
model = AutoModel.from_pretrained("../graphcodebert-base")

new_question_embeddings_768 = []
new_question_embeddings_32 = []
linear_proj = nn.Linear(768, 32)
for idx,question in tqdm(question_list.items()):
    # inputs = tokenizer(skill, return_tensors="pt", padding=True, truncation=False)
    truncated_code = question[:100]
    # 使用 tokenizer 处理数据，并填充到最大长度
    inputs = tokenizer(truncated_code, return_tensors="pt", padding="max_length", max_length=100, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        context_embeddings_768 = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        context_embeddings_32 = linear_proj(torch.tensor(context_embeddings_768).unsqueeze(0)).squeeze(0).numpy()
        # print(context_embeddings_768)
        # print(context_embeddings_32)
    new_question_embeddings_768.append(context_embeddings_768)
    new_question_embeddings_32.append(context_embeddings_32)

print(len(new_question_embeddings_768))
print(len(new_question_embeddings_32))



Some weights of RobertaModel were not initialized from the model checkpoint at ../graphcodebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/2233 [00:00<?, ?it/s]/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 2233/2233 [01:08<00:00, 32.51it/s]

2233
2233


In [8]:
import pickle
with open('../data/Python/questions_embeddings_index_768.pkl', 'wb') as f:
    pickle.dump(new_question_embeddings_768, f)
with open('../data/Python/questions_embeddings_index_32.pkl', 'wb') as f:
    pickle.dump(new_question_embeddings_32, f)

In [9]:
kc_df = pd.read_csv('../data/Python/all_concepts_python.csv')
kc_df.head()

,id,KC
0,1,String substring
1,2,Loop iteration
2,3,String concatenation
3,4,string slicing
4,5,String replacement


In [10]:
kc_name_dict = {}
for index, row in kc_df.iterrows():
    kc_name_dict[index] = row['KC']

In [11]:
from tqdm import tqdm
import torch
import torch.nn as nn
import pickle

new_kc_embeddings_768 = []
new_kc_embeddings_32 = []
linear_proj = nn.Linear(768, 32)
for idx,kc in tqdm(kc_name_dict.items()):
    # inputs = tokenizer(skill, return_tensors="pt", padding=True, truncation=False)
    truncated_code = kc[:15]
    # 使用 tokenizer 处理数据，并填充到最大长度
    inputs = tokenizer(truncated_code, return_tensors="pt", padding="max_length", max_length=15, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        context_embeddings_768 = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        context_embeddings_32 = linear_proj(torch.tensor(context_embeddings_768).unsqueeze(0)).squeeze(0).numpy()
        # print(context_embeddings_768)
        # print(context_embeddings_32)
    new_kc_embeddings_768.append(context_embeddings_768)
    new_kc_embeddings_32.append(context_embeddings_32)

print(len(new_kc_embeddings_768))
print(len(new_kc_embeddings_32))


with open('../data/Python/kc_embeddings_index_768.pkl', 'wb') as f:
    pickle.dump(new_question_embeddings_768, f)
with open('../data/Python/kc_embeddings_index_32.pkl', 'wb') as f:
    pickle.dump(new_question_embeddings_32, f)

  0%|          | 0/78 [00:00<?, ?it/s]/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 78/78 [00:02<00:00, 34.58it/s]

78
78


In [12]:
# 计算知识点的余弦相似度
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
kc_embeddings_768 = torch.tensor(new_kc_embeddings_768)
kc_similarity_768 = cosine_similarity(kc_embeddings_768)
threshold = 0.5
adj_matrix = (kc_similarity_768 > threshold).astype(np.float32)
adj_matrix.shape
np.save('../data/Python/adj_kc_codebert_codenet.npy', adj_matrix)

/var/folders/3p/44vy4f8j1sz8xbdfzyjvgdt80000gn/T/ipykernel_19672/3082362772.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  kc_embeddings_768 = torch.tensor(new_kc_embeddings_768)
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
